<a href="https://colab.research.google.com/github/Carocaro0424/Finanzas-Computacionales/blob/main/Programacion_lineal_taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SANTIAGO RODRIGUEZ VALENCIA (1010133588)-
PAULA CARO (1000049201)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import linprog


PUNTO 1 



Para formular el problema de Programación Lineal, primero definimos las variables de decisión:

> $x_A$: número de unidades del producto A producidas

> $x_B$: número de unidades del producto B producidas

Luego, se establecen las restricciones del problema:



*   Restricciones de recursos
1.   Materia prima:
      \begin{align}
      x_{A} + x_{B} \leq 500 m.p. \\
      \end{align}
2.   Mano de obra:
      \begin{align}
      2x_{A} + 4x_{B} \leq 1000 m.o. \\
      \end{align}
3.   Capital:
      \begin{align}
      x_{A} + 2x_{B} \leq 2000$ \\
      \end{align}
*    Restricciones de demanda
      \begin{align}
      x_{A} \geq 100 \\
      x_{B} \geq 150 \\
      \end{align}

*    Restricciones normales
      \begin{align}
      x_{A} \geq 0 \\
      x_{B} \geq 0 \\
      \end{align}
      
Finalmente, se establece la función objetivo que se desea maximizar:

$$\max Z = 3x_A + 5x_B$$


In [ ]:
from scipy.optimize import linprog

# Función objetivo
c = np.array([3, 5])

# Restricciones de recursos
A = np.array([[1, 1],[2, 4],[1, 2],[-1, 0],[0, -1]])
b = np.array([500, 1000, 2000,-100, -150])

# Restricciones de demanda
demanda = [[1, 0], [0, 1]]
disp_dem = [100, 150]

# Restricciones de no-negatividad
x0_bounds = (100, None)
x1_bounds = (150, None)

# Resolver problema de Programación Lineal
res = linprog(-c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds])

# Imprimir solución
print("Ganancia máxima: $", res.fun)
print("Cantidad producida de A: ", res.x[0]) # Esto debería ser 200?
print("Cantidad producida de B: ", res.x[1])

Ganancia máxima: $ -1350.0
Cantidad producida de A:  200.0
Cantidad producida de B:  150.0


PUNTO 2

Para formular el problema de LP, se deben definir las variables de decisión x1, x2, x3 y x4 como la cantidad invertida en cada fondo. La función objetivo se define como la suma ponderada de los retornos esperados de cada fondo, es decir,

Maximizar
\begin{align}
10x_{1} + 15x_{2} + 16x_{3} + 8x_{4}\\
\end{align}

Sujeto a las restricciones:

\begin{align}
0.5x_{1} + 0.3x_{2} + 0.25x_{3} + 0.6x_{4} \geq 0.35  \hspace{1cm} ( valor ) \\
0.3x_{1} + 0.1x_{2} + 0.4x_{3} + 0.2x_{4} \geq 0.30  \hspace{1cm} ( tamaño ) \\
0.2x_{1} + 0.6x_{2} + 0.35x_{3} + 0.3x_{4} \geq 0.15  \hspace{1cm} ( liquidez ) \\
x_{1} + x_{2} + x_{3} + x_{4} \equiv $100 \\
x_{1}, x_{2}, x_{3}, x_{4} \geq 0 \\
\end{align}

Aquí, las tres primeras restricciones aseguran que la asignación de fondos cumple con los requisitos de la estrategia de inversión (al menos el 35% en acciones de alto valor, el 30% en tamaño y el 15% en liquidez). La última restricción establece que el total invertido en los cuatro fondos debe ser de $100 millones.

Para resolver el problema de LP, se puede utilizar la librería CVXPY en Python. A continuación se muestra el código para resolver el problema:

In [ ]:
from binascii import a2b_base64
import cvxpy as cp
# Coeficientes de la función objetivo
c2 = np.array([0.10, 0.15, 0.16, 0.08])
A2 = np.array([[1,1,1,1]])
b2 = np.array([1])

D2 = np.array([[0.5, 0.3, 0.25, 0.6],
     [0.3, 0.1, 0.4, 0.2],
     [0.2, 0.6, 0.35, 0.2]])
d2 = np.array([0.35,0.30,0.15])


solution = linprog(-c2, A_eq=A2, b_eq=b2,A_ub=-D2,b_ub=-d2, method ="simplex")
pd.DataFrame((solution.x).round(4).T)*100




<ipython-input-7-a04cd645240d>:14: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  solution = linprog(-c2, A_eq=A2, b_eq=b2,A_ub=-D2,b_ub=-d2, method ="simplex")


,0
0,0.00
1,15.79
2,57.89
3,26.32


In [ ]:
import cvxpy as cp
n=4
x = cp.Variable(n)
Obj = cp.Minimize(-c2.T@x)
cons = [cp.sum(x)==1,D2@x>=d2,x>=0]
probl = cp.Problem(Obj,cons)
result = probl.solve()
x_sol2=pd.DataFrame(x.value)
pd.DataFrame((x_sol2).round(4).T)*100


,0,1,2,3
0,0.0,15.79,57.89,26.32



PUNTO 3




Para este caso debemos definir los coeficientes de la función objetivo y las restricciones. 

Restricciones: Restricciones de los saldos en cada mes y límites en xj, yj y zj

x1 + y1 - z1 = 150 

x2 + y2 - 1.01x1 + 1.003z1 -  z2 = 100

x3 + y3 - 1.01x2 + 1.003z2 -  z3 = -200

x4 - 1.02y1 – 1.01x3 + 1.003z3 -  z4 = 200

x5 - 1.02y2 -  1.01x4 + 1.003z4 -  z5 = -50

-1.02y3 – 1.01x5 + 1.003z5 – z6 = -300

xj <= 100 para j = 1, 2, 3, 4, 5

xj >= 0 para j = 1, 2, 3, 4, 5

yj >= 0 para j = 1, 2, 3

zj >= 0 para j = 1, 2, 3, 4, 5

En este caso, la función objetivo es maximizar el saldo de fondos en el mes 6, es decir, z6

In [ ]:
c3= np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1])
A3 = np.array([[1,0,0,0,0,1,0,0,-1,0,0,0,0,0],
               [-1.01,1,0,0,0,0,1,0,1.003,-1,0,0,0,0],
                [0,-1.01,1,0,0,0,0,1,0,1.003,-1,0,0,0],
               [0,0,-1.01,1,0,-1.02,0,0,0,0,1.003,-1,0,0],
               [0,0,0,-1.01,1,0,-1.02,0,0,0,0,1.003,-1,0],
               [0,0,0,0,-1.01,0,0,-1.02,0,0,0,0,1.003,-1]])

b3 = np.array([150, 100, -200, 200, -50, -300 ])

A3_ub = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1]])

b3_ub = np.array([100])

res = linprog(-c3, A_ub=-A3_ub, b_ub=-b3_ub, A_eq=A3, b_eq=b3,
              
              method='revised simplex') 
res
res = linprog(-c3, A_ub=A3, b_ub=b3, bounds=bounds, method='simplex')
pd.DataFrame(res.x).round() 

<ipython-input-37-b6fd8f98df37>:15: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(-c3, A_ub=-A3_ub, b_ub=-b3_ub, A_eq=A3, b_eq=b3,
<ipython-input-37-b6fd8f98df37>:19: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(-c3, A_ub=A3, b_ub=b3, bounds=bounds, method='simplex')


,0
0,0.0
1,0.0
2,100.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,100.0
9,0.0


PUNTO 4 





El objetivo es maximizar la rentabilidad total de los saldos en la cuenta de la empresa. La rentabilidad se deriva de la inversión en un instrumento a corto plazo con un rendimiento del 0,02% por día. Además, hay un costo de transacción del 0,01% del monto transferido al realizar una inversión.

Las restricciones son las siguientes:

La empresa debe mantener un saldo suficiente en su cuenta para cubrir los pagos necesarios durante el mes.
La empresa debe asegurarse de que los flujos de efectivo entren y salgan de la cuenta en las semanas apropiadas.
La solución del problema de optimización debe determinar la cantidad de dinero que la empresa debe invertir cada día en el instrumento a corto plazo para maximizar su rentabilidad total.

A continuación, se presenta la formulación matemática del problema de LP:

Variables de decisión:

x_i: cantidad de dinero que se invierte en el instrumento a corto plazo en el día i (i = 1, ..., 28).

Función objetivo:

Maximizar la rentabilidad total de los saldos en la cuenta:

Max Z = sum(x_i * 0.02% - x_i * 0.01%)

Restricciones:

La empresa debe mantener un saldo suficiente para cubrir los pagos necesarios durante el mes:

x_1 + 500 >= 350

x_8 + x_9 + x_10 + 300 >= 400

x_15 + x_16 + x_17 + x_18 + 200 >= 250

x_22 + x_23 + x_24 + x_25 + x_26 + x_27 + x_28 + 400 >= 200

La empresa debe asegurarse de que los flujos de efectivo entren y salgan de la cuenta en las semanas apropiadas:

x_1 + 500 = x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8

x_8 + x_9 + x_10 + 300 = x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18

x_15 + x_16 + x_17 + x_18 + 200 = x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 + x_27 + x_28 + 350

Las variables de decisión deben ser no negativas:

x_i >= 0 (i = 1, ..., 28)



In [ ]:
import numpy as np
from scipy.optimize import linprog

# Función objetivo
c4 = np.array([0,0,0,0,0,0,0,1]) # Maximizar rentabilidad
r= (1+0.0001)**7
#Restricciones
A4 = np.array([[1,0,0,0,-1,0,0,0],
              [0,1,0,0, r,-1,0,0],
              [0,0,1,0,0,r,-1,0],
              [0,0,0,1,0,0,r,-1]])
    
b4 = np.array([[150,-100,-50,200]])

bounds = [(0, None), 
          (0, None), 
          (0, None), 
          (0, None), 
          (0, None), 
          (0, None), 
          (0, None), 
          (0, None)]

# Resolver el problema de programación lineal
res = linprog(-c4, A_ub=A4, b_ub=-b4, bounds=bounds, method='simplex')
pd.DataFrame(res.x).round(4).T

#print(res)

<ipython-input-23-a613fae73839>:25: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(-c4, A_ub=A4, b_ub=-b4, bounds=bounds, method='simplex')


,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,150.0,50.105,0.1401,200.1402


PUNTO 5

La empresa XYZ tiene que decidir como distribuye su presupuesto $10.000 millones y la
mano de obra (horas de trabajo) en tres departamentos: ventas, marketing y producción
buscando maximizar sus ganancias. Para ello, se sabe que cada departamento le
proporciona a la empresa las siguientes ganancias: ventas: $5, marketing: 8$, y
producción: $10. Esta asignación debe cumplir las siguientes restricciones:
i. El total de horas de trabajo disponibles es de 1.000.
ii. El departamento de ventas debe recibir al menos el 20% del presupuesto total.
iii. El departamento de marketing debe recibir al menos el 30% del total de horas de
trabajo disponibles.
iv. El departamento de producción debe recibir al menos el 25% del total de horas de
trabajo disponibles.
Formule un problema de LP utilizando Scipy que le permita maximizar

In [71]:
c5= np.array([0,0,0,2,8,10])
A5= ([0,0,0,1,0,0],
     [0,1,0,0,0,0],
     [0,0,1,0,0,0],
     [0,0,0,0,1,0],
     [0,0,0,0,0,1])
b5 = ([2000,300,250,2000,2000])
# Restricciones de igualdad
A_eq5 = np.array([[1,1,1,0,0,0],[0,0,0,1,1,1]])
B_eq5 = np.array([1000,10000])   
# Resolver el problema de programación lineal
res = linprog(-c5, A_ub=A5, b_ub=b5,A_eq=A_eq5,b_eq=B_eq5, method='simplex')
pd.DataFrame(res.x).round(4).T             


<ipython-input-71-0e0c4b906f6e>:12: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(-c5, A_ub=A5, b_ub=b5,A_eq=A_eq5,b_eq=B_eq5, method='simplex')


,0,1,2,3,4,5
0,450.0,300.0,250.0,2000.0,2000.0,2000.0


PUNTO 6

Considere una empresa con tres puntos potenciales para ubicar sus instalaciones (J =
{1,2,3}) y cinco puntos de demanda que debe satisfacer (I = {1,2,3,4,5}). La apertura del
punto j ∈ J tiene un costo fijo de funcionamiento fj

, independientemente del volumen de
ventas que maneje. Este volumen está limitado a una determinada cantidad máxima que
puede manejarse anualmente (Mj

). Además, hay un costo de transporte cij para ir desde el
punto j para abastecer la demanda en el punto i. Las siguientes tablas muestran los datos
de demanda en cada punto:

In [72]:
# Coeficientes de la funcion objetivo
c6 = np.array([[4,5,6,8,10],
              [6,4,3,5,8],
              [9,7,4,3,4]])
# Restricciones de produccion 
Restb6 = np.array([500,500,500])
M6 = np.array([1000,1000,1000])
Funcióndemanda6= np.array([80,270,250, 160, 180])

# Minimizar la ganancia, en base a las restricciones
x= cp.Variable((3,5), nonneg=True)
y= cp.Variable(3, boolean=True)
objetivo= cp.Minimize(cp.sum(cp.multiply(c6,x)))

Restricciones= [cp.sum(x,axis=1)<=M6 @ y, cp.sum(x,axis=0)==Funcióndemanda6, cp.sum(y*M6)>=cp.sum(Funcióndemanda6)]

Prob = cp.Problem(objetivo, Restricciones)
Prob.solve()
x.value

/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


array([[ 80.,  -0.,  -0.,  -0.,  -0.],
       [ -0., 270., 250.,  -0.,  -0.],
       [ -0.,  -0.,  -0., 160., 180.]])